In [1]:
import torch

In [2]:
cuda = torch.device('cuda')     # Default CUDA device
cuda0 = torch.device('cuda:0')  # GPU 0 (these are 0-indexed)
cuda1 = torch.device('cuda:1')  # GPU 1 (these are 0-indexed)

In [3]:
x = torch.tensor([1., 2.], device=cuda0)
# x.device is device(type='cuda', index=0)
y = torch.tensor([1., 2.]).cuda()
# y.device is device(type='cuda', index=0)

with torch.cuda.device(1):
    # allocates a tensor on GPU 1
    a = torch.tensor([1., 2.], device=cuda)

    # transfers a tensor from CPU to GPU 1
    b = torch.tensor([1., 2.]).cuda()
    # a.device and b.device are device(type='cuda', index=1)

    print('a', a.device)
    print('b', b.device)

    # You can also use ``Tensor.to`` to transfer a tensor:
    b2 = torch.tensor([1., 2.]).to(device=cuda)
    # b.device and b2.device are device(type='cuda', index=1)

    print('b2', b2.device)

    c = a + b
    # c.device is device(type='cuda', index=1)

    print('c', c.device)

    z = x + y
    # z.device is device(type='cuda', index=0)

    print('z', z.device)
    
    # even within a context, you can specify the device
    # (or give a GPU index to the .cuda call)
    d = torch.randn(2, device=cuda0)
    e = torch.randn(2).to(cuda0)
    f = torch.randn(2).cuda(cuda0)
    # d.device, e.device, and f.device are all device(type='cuda', index=2)

    print('d', d.device)
    print('e', e.device)
    print('f', f.device)

a cuda:1
b cuda:1
b2 cuda:1
c cuda:1
z cuda:0
d cuda:0
e cuda:0
f cuda:0
